# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
peter = Website("https://www.peterleeds.com/")
peter.links

['/',
 '/',
 '/about-peter-leeds.htm',
 '/in-the-media.htm',
 '/peters-message.htm',
 'https://www.youtube.com/user/PeterLeedsPennyStock',
 '/our-team.htm',
 '/testimonials.htm',
 '/leeds-analysis.htm',
 '/step-by-step-penny-stock-guide.htm',
 '/what-are-penny-stocks.htm',
 '/faq.htm',
 '/100-unbiased.htm',
 '/infographics.htm',
 '/insiders.htm',
 '/how-it-works.htm',
 '/academy.htm',
 '/candlestick.htm ',
 '/up.thinking.htm',
 '/penny-stocks-book.htm',
 '/track-record.htm',
 '/contact-us.htm',
 '/legal.htm',
 'subscribe.htm',
 '/',
 '/about-peter-leeds.htm',
 '/in-the-media.htm',
 '/peters-message.htm',
 'https://www.youtube.com/user/PeterLeedsPennyStock',
 '/our-team.htm',
 '/testimonials.htm',
 '/leeds-analysis.htm',
 '/step-by-step-penny-stock-guide.htm',
 '/what-are-penny-stocks.htm',
 '/faq.htm',
 '/100-unbiased.htm',
 '/infographics.htm',
 '/insiders.htm',
 '/how-it-works.htm',
 '/academy.htm',
 '/candlestick.htm ',
 '/up.thinking.htm',
 '/penny-stocks-book.htm',
 '/track-record

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(peter))

Here is the list of links on the website of https://www.peterleeds.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/
/about-peter-leeds.htm
/in-the-media.htm
/peters-message.htm
https://www.youtube.com/user/PeterLeedsPennyStock
/our-team.htm
/testimonials.htm
/leeds-analysis.htm
/step-by-step-penny-stock-guide.htm
/what-are-penny-stocks.htm
/faq.htm
/100-unbiased.htm
/infographics.htm
/insiders.htm
/how-it-works.htm
/academy.htm
/candlestick.htm 
/up.thinking.htm
/penny-stocks-book.htm
/track-record.htm
/contact-us.htm
/legal.htm
subscribe.htm
/
/about-peter-leeds.htm
/in-the-media.htm
/peters-message.htm
https://www.youtube.com/user/PeterLeedsPennyStock
/our-team.htm
/testimonials.htm
/leeds-analysis.htm
/step-by-step-penny-stock-guide.htm
/what-are-penny-stocks.htm
/faq.htm
/100-unbiased.htm
/infogr

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/SRPO',
 '/Qwen/Qwen3-Next-80B-A3B-Instruct',
 '/openbmb/VoxCPM-0.5B',
 '/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B',
 '/google/vaultgemma-1b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/IndexTeam/IndexTTS-2-Demo',
 '/spaces/multimodalart/wan-2-2-first-last-frame',
 '/spaces/abdul9999/NoWatermark',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/InternRobotics/OmniWorld',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 

In [13]:
get_links("https://www.peterleeds.com/")

{'links': [{'type': 'about page',
   'url': 'https://www.peterleeds.com/about-peter-leeds.htm'},
  {'type': 'our team page', 'url': 'https://www.peterleeds.com/our-team.htm'},
  {'type': 'testimonials page',
   'url': 'https://www.peterleeds.com/testimonials.htm'},
  {'type': 'how it works page',
   'url': 'https://www.peterleeds.com/how-it-works.htm'},
  {'type': 'academy page', 'url': 'https://www.peterleeds.com/academy.htm'},
  {'type': 'contact us page',
   'url': 'https://www.peterleeds.com/contact-us.htm'},
  {'type': 'penny stocks book page',
   'url': 'https://www.peterleeds.com/penny-stocks-book.htm'},
  {'type': 'media page', 'url': 'https://www.peterleeds.com/in-the-media.htm'},
  {'type': 'track record page',
   'url': 'https://www.peterleeds.com/track-record.htm'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/SRPO
Updated
4 days ago
•
5.23k
•
821
Qwen/Qwen3-Next-80B-A3B-Instruct
Updated
2 days ago
•
408k
•
668
openbmb/VoxCPM-0.5B
Updated
about 19 hours ago
•
393
•
37

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/SRPO\nUpdated\n4 days ago\n•\n5.23k\n•\n821\nQwen/Qwen3-Next-80B-A3B-Instruct\nUpdated\n2 days ago\n•\n408k\n•\n668\nopenbmb/VoxCPM-0.5B\nUpdated\nabout 19 hours ago\n•\n393\n•\n376\nAlibaba-NLP/Tongyi-DeepResearch-30B-A3B\nUpdated\n2 days ago\n•\n1.03k\n•\n371\ngoogle/vaultgemma-1b\nUpdated\n7 days ago\n•\n2.07k\n•\n336\nBrowse 1M+ models\nSpaces\nRunning\n13.8k\n13.8k\nDeepSite v

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face Brochure

## About Us

**Hugging Face** is an innovative platform that serves as a hub for the machine learning community, fostering collaboration on models, datasets, and applications. We are committed to building the future of AI together, emphasizing open-source and community-driven development.

### Our Mission
We aim to create a space where machine learning professionals and enthusiasts can **explore**, **collaborate**, and **innovate**. With over **1 million models** and **250,000 datasets**, our platform is engineered to enhance productivity and capabilities in the AI ecosystem.

## Company Culture

At Hugging Face, we pride ourselves on fostering a **collaborative and inclusive community**. Our culture is built around:

- **Innovation**: Encouraging creative solutions and the exploration of new ideas within the AI community.
- **Openness**: Promoting transparency in our processes and projects by supporting open-source initiatives.
- **Support**: Providing resources for individuals and teams to thrive—whether through documentation, community engagement, or enterprise support.

### Community Engagement
We have built a robust community featuring contributions from over **50,000 organizations**, including industry leaders like Google, Amazon, Meta, and Microsoft. Our community spaces include forums, collaborative projects, and shared learning resources.

## Products and Services

### For Developers
- **Models**: Browse and collaborate on **1M+ AI models** with state-of-the-art capabilities.
- **Datasets**: Access a diverse array of **250k+ datasets** crucial for training and evaluating models.
- **Spaces**: Explore practical applications and create AI applications using our dedicated hosting space.

### For Enterprises
We offer enterprise-grade solutions that include:
- **Advanced Security Features**: Single Sign-On, priority support, and dedicated resource groups.
- **Customizable Solutions**: Tailored APIs and services to fit your organizational needs.

### Pricing
Affordable compute solutions start at **$0.60/hour for GPU** usage and enterprise team solutions starting at **$20/user/month**.

## Career Opportunities

Join a leading team that is shaping the future of AI! We are constantly in search of passionate individuals who want to make an impact in the field of machine learning. Current openings span a variety of roles in engineering, product management, and community engagement.

Explore our career page for the latest job openings and become a part of the Hugging Face team!

## Get Involved

Whether you're a developer, a researcher, or someone who is passionate about AI, **Hugging Face** has something for you! 

- **Explore AI Applications**: Check out the latest trending models and datasets.
- **Join Our Community**: Engage with fellow enthusiasts on platforms like Discord and GitHub.
- **Contribute**: Share your own models, datasets, or ideas with the world.

### Connect with Us

- [Visit our Website](https://huggingface.co)
- Follow us on [Twitter](https://twitter.com/huggingface) and [LinkedIn](https://www.linkedin.com/company/huggingface).

---

Join us at Hugging Face and be a part of the AI revolution!
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'support page', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## **About Us**
Hugging Face is the leading AI community dedicated to building the future of machine learning. We provide a collaborative platform where individuals and organizations come together to share and develop cutting-edge models, datasets, and applications. Our community-driven approach fosters innovation and accelerates the adoption of AI technologies.

## **Our Platform**
### **Models**
Join a thriving ecosystem with over **1 million models** to choose from. From the latest innovations in Natural Language Processing to state-of-the-art visual recognition models, find the solution that fits your needs.

### **Datasets**
Access a vast collection of **250,000+ datasets**, enabling seamless integration for any machine learning task. Our resources empower researchers and organizations to push the boundaries of what AI can achieve.

### **Spaces**
Explore **400,000+ applications** in various formats, ranging from interactive web demos to deployed production models. Benefit from user-generated content that showcases the versatility of machine learning in real-world applications.

## **Our Culture**
At Hugging Face, we value **collaboration, openness, and innovation**. Our friendly community of data scientists, engineers, and researchers work in unison to share knowledge and creativity. We believe that great breakthroughs come from partnership, enabling individuals to build upon each other's work and accelerate AI advancements.

## **Who We Serve**
More than **50,000 organizations** trust Hugging Face to enhance their AI endeavors. Esteemed companies including Google, Microsoft, Amazon, and Intel utilize our platform's vast resources. We cater to both startups and established enterprises alike, ensuring that anyone can harness the power of machine learning.

## **Careers at Hugging Face**
We are always looking for passionate and talented individuals to join our team. If you aspire to contribute to the future of AI, explore our current job openings [here](https://huggingface.co/jobs). We offer:
- **Flexible Work Environment:** Embrace remote work and a diverse culture.
- **Collaborative Teams:** Engage in knowledge-sharing and mutual growth.
- **Growth Opportunities:** Focus on continuous learning and professional development.

## **Join Us!**
Become part of the **AI community building the future**. Together, we can explore limitless possibilities in machine learning and drive transformative change. Sign up today and start your journey with Hugging Face!

**Learn more at** [Hugging Face](https://huggingface.co)  
**Follow us on Social Media:**
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [GitHub](https://github.com/huggingface)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>